#### CSC 296S Deep Learning (Spring 2026)

#### Dr. Haiquan Chen, Dept of Computer Scicence

#### California State University, Sacramento




## Lab 3: Evaluating Neural Networks 


### Helpful Functions for Tensorflow (little gems)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network. 

* Predictors/Inputs 
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.

In [1]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


# Training with a Test Set with Early Stopping

**Overfitting** occurs when a neural network is trained to the point that it begins to memorize rather than generalize.  




### Split data into training and test using train_test_split

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()

model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

# patience: number of epochs with no improvement after which training will be stopped

# The test set is checked during training to monitor progress for early stopping but is never used for gradient descent (model training)

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor], verbose=2, epochs=1000)  


Epoch 1/1000
4/4 - 3s - loss: 2.1299 - val_loss: 1.9073 - 3s/epoch - 700ms/step
Epoch 2/1000
4/4 - 0s - loss: 1.9863 - val_loss: 1.7828 - 36ms/epoch - 9ms/step
Epoch 3/1000
4/4 - 0s - loss: 1.8481 - val_loss: 1.6688 - 37ms/epoch - 9ms/step
Epoch 4/1000
4/4 - 0s - loss: 1.7164 - val_loss: 1.5655 - 30ms/epoch - 7ms/step
Epoch 5/1000
4/4 - 0s - loss: 1.6064 - val_loss: 1.4714 - 39ms/epoch - 10ms/step
Epoch 6/1000
4/4 - 0s - loss: 1.4961 - val_loss: 1.3894 - 43ms/epoch - 11ms/step
Epoch 7/1000
4/4 - 0s - loss: 1.4005 - val_loss: 1.3180 - 42ms/epoch - 10ms/step
Epoch 8/1000
4/4 - 0s - loss: 1.3217 - val_loss: 1.2564 - 40ms/epoch - 10ms/step
Epoch 9/1000
4/4 - 0s - loss: 1.2475 - val_loss: 1.2052 - 38ms/epoch - 9ms/step
Epoch 10/1000
4/4 - 0s - loss: 1.1910 - val_loss: 1.1632 - 43ms/epoch - 11ms/step
Epoch 11/1000
4/4 - 0s - loss: 1.1413 - val_loss: 1.1306 - 39ms/epoch - 10ms/step
Epoch 12/1000
4/4 - 0s - loss: 1.1045 - val_loss: 1.1059 - 41ms/epoch - 10ms/step
Epoch 13/1000
4/4 - 0s - loss:

Now that the neural network is trained, we can make predictions about the test set.  The following code predicts the type of iris for test set and displays the first five irises. 

In [3]:
pred = model.predict(x_test)
print(pred[0:5]) # print first five predictions

2/2 [==============================] - 0s 3ms/step
[[2.5735209e-03 8.4674406e-01 1.5068233e-01]
 [9.9383461e-01 3.3801217e-03 2.7853858e-03]
 [1.0224261e-06 1.1324255e-02 9.8867470e-01]
 [3.3346717e-03 6.7597693e-01 3.2068843e-01]
 [1.5252521e-03 8.1747562e-01 1.8099910e-01]]


Each line provides the probability that the iris is one of the 3 types of iris in the data set. 

## Saving Best Weights

It would be good idea to keep track of the most optimal weights during the entire training operation.  


An additional monitor, ModelCheckpoint,  is used and saves a copy of the neural network to **best_weights.hdf5** each time the validation score of the neural network improves.  

Once training is done, we just reload this file and we have the optimal training weights that were found.

In [4]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")
x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

checkpointer = ModelCheckpoint(filepath="dnn/best_weights.keras", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)

model.load_weights('dnn/best_weights.keras') # load weights from best model

Epoch 1/1000
4/4 - 0s - loss: 2.4902 - val_loss: 2.4488 - 399ms/epoch - 100ms/step
Epoch 2/1000
4/4 - 0s - loss: 2.3377 - val_loss: 2.2917 - 65ms/epoch - 16ms/step
Epoch 3/1000
4/4 - 0s - loss: 2.1854 - val_loss: 2.1454 - 64ms/epoch - 16ms/step
Epoch 4/1000
4/4 - 0s - loss: 2.0471 - val_loss: 2.0090 - 59ms/epoch - 15ms/step
Epoch 5/1000
4/4 - 0s - loss: 1.9149 - val_loss: 1.8847 - 54ms/epoch - 14ms/step
Epoch 6/1000
4/4 - 0s - loss: 1.7932 - val_loss: 1.7727 - 58ms/epoch - 15ms/step
Epoch 7/1000
4/4 - 0s - loss: 1.6912 - val_loss: 1.6715 - 61ms/epoch - 15ms/step
Epoch 8/1000
4/4 - 0s - loss: 1.5993 - val_loss: 1.5812 - 60ms/epoch - 15ms/step
Epoch 9/1000
4/4 - 0s - loss: 1.5158 - val_loss: 1.5060 - 76ms/epoch - 19ms/step
Epoch 10/1000
4/4 - 0s - loss: 1.4407 - val_loss: 1.4390 - 50ms/epoch - 13ms/step
Epoch 11/1000
4/4 - 0s - loss: 1.3831 - val_loss: 1.3795 - 59ms/epoch - 15ms/step
Epoch 12/1000
4/4 - 0s - loss: 1.3274 - val_loss: 1.3292 - 58ms/epoch - 14ms/step
Epoch 13/1000
4/4 - 0s 

## Evaluating Classification Models

### (1) Calculate Classification Accuracy/Precision/Recall/F1-Score

By default, Keras will return the predicted probability for each class. We can change these prediction probabilities into the actual iris predicted with **argmax**.

In [5]:
pred = model.predict(x_test)
pred

2/2 [==============================] - 0s 3ms/step


array([[3.02068610e-03, 8.18546772e-01, 1.78432539e-01],
       [9.97824669e-01, 2.11860938e-03, 5.66952549e-05],
       [1.53437998e-06, 1.36551959e-02, 9.86343265e-01],
       [3.83890187e-03, 8.04279804e-01, 1.91881210e-01],
       [1.80173526e-03, 8.65325034e-01, 1.32873267e-01],
       [9.95995760e-01, 3.89426691e-03, 1.09944485e-04],
       [2.25135945e-02, 9.46763456e-01, 3.07229646e-02],
       [4.53560089e-04, 3.38461906e-01, 6.61084533e-01],
       [1.07385102e-03, 4.99004990e-01, 4.99921083e-01],
       [9.07012261e-03, 9.58573520e-01, 3.23563926e-02],
       [8.82567430e-04, 4.26781356e-01, 5.72336137e-01],
       [9.95365143e-01, 4.44048177e-03, 1.94392240e-04],
       [9.98854995e-01, 1.11603620e-03, 2.88787851e-05],
       [9.95157540e-01, 4.64386353e-03, 1.98636204e-04],
       [9.98750806e-01, 1.19659991e-03, 5.26836957e-05],
       [4.07943456e-03, 8.82686555e-01, 1.13233961e-01],
       [2.15685559e-05, 2.84787323e-02, 9.71499622e-01],
       [8.45088344e-03, 9.34801

In [6]:
pred = np.argmax(pred,axis=1) # raw probabilities to choose class (highest probability)
print(pred)

[1 0 2 1 1 0 1 2 2 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


Now that we have the actual iris flower predicted, we can calculate the percent accuracy (how many were correctly classified).

In [7]:
y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

Accuracy score: 0.9736842105263158


In [8]:
score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

Precision score: 0.9757085020242916


In [9]:
score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

Recall score: 0.9736842105263158


In [10]:
score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

F1 score: 0.9735839598997494


In [11]:
print(metrics.classification_report(y_true, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      0.91      0.95        11
           2       0.92      1.00      0.96        12

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38



### (2) Calculate Classification Cross-Entropy Loss (Log Loss)  

Log loss is an error metric that is often used in place of accuracy for classification.  

Log loss allows for "partial credit". For example, a model might be used to classify A, B and C.  The correct answer might be A, however if the classification network chose B as having the highest probability, then accuracy gives the neural network no credit for this classification.  

However, with log loss, the probability of the correct answer is added to the score.  For example, the correct answer might be A, but if the neural network only predicted .4 probability of A being correct, then the value -log(.4) is added.

$$ logloss = -\frac{1}{N}\sum^N_{i=1}\sum^M_{j=1}y_{ij} \log(\hat{y}_{ij}) $$

The following code shows the logloss scores that correspond to the average probablity for the correct item. The **pred** column specifies the average robability for the correct class.  The **logloss** column specifies the log loss for that probability.


Calculating log loss

In [12]:
# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
print(pred[0:5])
print()
print("y_test:")
print(y_test[0:5])

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

2/2 [==============================] - 0s 3ms/step
Numpy array of predictions
[[3.0206861e-03 8.1854677e-01 1.7843254e-01]
 [9.9782467e-01 2.1186094e-03 5.6695255e-05]
 [1.5343800e-06 1.3655196e-02 9.8634326e-01]
 [3.8389019e-03 8.0427980e-01 1.9188121e-01]
 [1.8017353e-03 8.6532503e-01 1.3287327e-01]]

y_test:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
Log loss score: 0.11683332206627528


## Evaluating Regression Models

Regression results are evaluated differently than classification.  Consider the following code. 

In [14]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import metrics

path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']
df.drop('name',axis=1,inplace=True)
missing_median(df, 'horsepower')

encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.25, random_state=45)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10))
model.add(Dense(10))
model.add(Dense(10))

model.add(Dense(1))  # 1 output neuron 


model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.fit(x_train,y_train, validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
10/10 - 1s - loss: 1179510.6250 - val_loss: 456151.5312 - 530ms/epoch - 53ms/step
Epoch 2/1000
10/10 - 0s - loss: 211200.0938 - val_loss: 24256.4746 - 64ms/epoch - 6ms/step
Epoch 3/1000
10/10 - 0s - loss: 6420.0825 - val_loss: 13886.6826 - 101ms/epoch - 10ms/step
Epoch 4/1000
10/10 - 0s - loss: 20710.9180 - val_loss: 20135.8379 - 55ms/epoch - 6ms/step
Epoch 5/1000
10/10 - 0s - loss: 12049.1367 - val_loss: 2981.2588 - 57ms/epoch - 6ms/step
Epoch 6/1000
10/10 - 0s - loss: 968.2250 - val_loss: 484.0277 - 67ms/epoch - 7ms/step
Epoch 7/1000
10/10 - 0s - loss: 921.6419 - val_loss: 993.5455 - 66ms/epoch - 7ms/step
Epoch 8/1000
10/10 - 0s - loss: 704.2963 - val_loss: 253.3685 - 71ms/epoch - 7ms/step
Epoch 9/1000
10/10 - 0s - loss: 224.5983 - val_loss: 210.1774 - 61ms/epoch - 6ms/step
Epoch 10/1000
10/10 - 0s - loss: 258.2898 - val_loss: 210.5182 - 60ms/epoch - 6ms/step
Epoch 11/1000
10/10 - 0s - loss: 224.7515 - val_loss: 165.5391 - 56ms/epoch - 6ms/step
Epoch 12/1000
10/10 - 0s -

### Mean Square Error

The mean square error is the sum of the squared differences between the prediction ($\hat{y}$) and the expected ($y$).  MSE values are not of a particular unit.  If an MSE value has decreased for a model, that is good. Low MSE values are desired.

$ \text{MSE} = \frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2 $


In [15]:
# Predict
pred = model.predict(x_test)

# Measure MSE error.  
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

4/4 [==============================] - 0s 4ms/step
Final score (MSE): 162.90277099609375


### Root Mean Square Error

The root mean square (RMSE) is essentially the square root of the MSE.  Because of this, the RMSE error is in the same units as the training data outcome. Low RMSE values are desired.

$ \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2} $

In [16]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 12.763337135314941


# Performance Improvement by Normalizing Features and Tuning Hyperparameters

There are many different settings that you can use for a neural network.  These can affect performance.  The following code changes some of these, beyond their default values:

* **activation:** relu, sigmoid, tanh
* **Layers and Neuron Counts**
* **optimizer:** adam, sgd, rmsprop, and [others](https://keras.io/optimizers/)

In [22]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
preprocess = True

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# create feature vector
missing_median(df, 'horsepower')
encode_text_dummy(df, 'origin')
df.drop('name',axis=1,inplace=True)

if preprocess:
    encode_numeric_zscore(df, 'horsepower')
    encode_numeric_zscore(df, 'weight')
    encode_numeric_zscore(df, 'cylinders')
    encode_numeric_zscore(df, 'displacement')
    encode_numeric_zscore(df, 'acceleration')
    encode_numeric_zscore(df, 'year')

# Encode to a 2D matrix for training
x,y = to_xy(df,'mpg')

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')  # change to sgd

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
10/10 - 0s - loss: 604.1018 - val_loss: 555.9454 - 389ms/epoch - 39ms/step
Epoch 2/1000
10/10 - 0s - loss: 567.9280 - val_loss: 512.5710 - 64ms/epoch - 6ms/step
Epoch 3/1000
10/10 - 0s - loss: 513.7374 - val_loss: 444.8324 - 63ms/epoch - 6ms/step
Epoch 4/1000
10/10 - 0s - loss: 428.1823 - val_loss: 341.7850 - 66ms/epoch - 7ms/step
Epoch 5/1000
10/10 - 0s - loss: 302.8359 - val_loss: 201.7024 - 59ms/epoch - 6ms/step
Epoch 6/1000
10/10 - 0s - loss: 151.4020 - val_loss: 74.0189 - 64ms/epoch - 6ms/step
Epoch 7/1000
10/10 - 0s - loss: 48.5382 - val_loss: 36.8976 - 65ms/epoch - 6ms/step
Epoch 8/1000
10/10 - 0s - loss: 36.5310 - val_loss: 37.0628 - 62ms/epoch - 6ms/step
Epoch 9/1000
10/10 - 0s - loss: 29.8039 - val_loss: 24.7186 - 63ms/epoch - 6ms/step
Epoch 10/1000
10/10 - 0s - loss: 23.2229 - val_loss: 21.5957 - 65ms/epoch - 6ms/step
Epoch 11/1000
10/10 - 0s - loss: 20.7217 - val_loss: 18.6871 - 61ms/epoch - 6ms/step
Epoch 12/1000
10/10 - 0s - loss: 18.3877 - val_loss: 16.4932 

In [23]:
# Predict and measure RMSE
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

3/3 [==============================] - 0s 4ms/step
Score (RMSE): 2.3167121410369873


In [25]:
# print out prediction
df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(pred, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

,ground_truth,predicted
0,33.0,33.355316
1,28.0,31.210648
2,19.0,20.881659
3,13.0,15.180898
4,14.0,12.992521
...,...,...
75,19.9,21.646719
76,17.5,17.926146
77,28.0,31.076693
78,29.0,30.402113
